# Cling Magic!
First, install the cling magic

In [ ]:
from cling import cling

Now you can execute C++ statements, one at a time. Usually, it's a
good idea to execute headers by themselves.

In [ ]:
%%cling
#include <iostream>

When you provide a cling magic cell, it starts a background cling interpreter. All subsequent
uses of cling magic sends commands to that same interpreter.

In [ ]:
%%cling
std::cout << "magic!" << std::endl;

## Communicating between Python and Cling
If you define Python variables, you can send them to your c++ code. Anywhere
the string {pythonvar} occurs, the C++ code will be replaced with the contents of the Python variable.

In [ ]:
foo = "world"

In [ ]:
%%cling
// Note that "baz" is not defined by Python, but "foo" is.
std::cout << "Hello {foo} {baz}" << std::endl;

You can also communicate from C++ to Python. To do this, provide a variable name
on the same line as the "%%cling" statement.

In [ ]:
%%cling feedback
int a;
a = 3;

In [ ]:
feedback

## Replay
It may happen that you kill your session by causing a segfault.

In [ ]:
%%cling
int *ptr = 0;
ptr[0]=1;

In this case, you can replay all the commands prior to the segfault.

In [ ]:
from cling import replay
replay()

In [ ]:
%%cling
std::cout << "a=" << a << std::endl;

In [ ]:
%%cling --init
#include <run_hpx.cpp>
#include <functional>

In [ ]:
%%cling
hpx::future<int> fib(int n) {
    if(n < 2) return hpx::make_ready_future(n);
    hpx::future<int> f1 = hpx::async(fib,n-1);
    hpx::future<int> f2 = fib(n-2);
    return hpx::dataflow(hpx::util::unwrapping([](int f1_, int f2_) {
        return f1_+f2_;
    }), f1, f2);
}
run_hpx([](){
    int n=10;
    std::cout << "fib(" << n << ")=" << fib(n).get() << std::endl;
});

In [ ]:
%%cling
run_hpx([](){
   auto f = hpx::async([](){ return 42; });
   std::cout << "f=" << f.get() << std::endl;
});

In [ ]:
%%cling
auto f2 = run_hpx([](){ return hpx::async([](){ return 42; }); });
run_hpx([&](){ std::cout << "f=" << f2.get() << std::endl; });